In [ ]:
import xarray as xr
import numpy as np
import hvplot as hv
import matplotlib.pyplot as plt

In [ ]:
file = 'erdHadISST_2deg_EPac.nc'
ds = xr.open_dataset(file)

In [ ]:
# get dimensions - for reshaping 3-D to 2-D data matrix
nt,nlat,nlon=ds.sst.shape
ds.sst.shape

In [ ]:
# quick look sanity check
ds.sst.isel(time=100).plot()

In [ ]:
ds.sst.isel(latitude=0).plot()

In [ ]:
# We see that points on land are all NaN
# These must be eliminated before we compute covariance or SVD. 
# Setting them to zero will suffice for now
data = ds.sst.fillna(0)
data[:,2,:].plot()
# nans are replaced with zeros

In [ ]:
# Recompute the mean
datam=data.mean(axis=0)
datam.plot()

In [ ]:
# Remove the mean from each time series so we can work with covariances
datap=data-datam
datap[:,6,:].plot()
# Looks like El Niño (the land values are zero)

In [ ]:
# Create 2-D data matrix. Time is row dimension; position is column dimension
datap2d=np.reshape(datap.values,((nt,-1)))
datap2d.shape

In [ ]:
# The correct order of transpose operations to get covariance over time:
C=np.dot(datap2d.T,datap2d)

In [ ]:
# EOFs are eigenvectors of the covariance matrix
eigvals,eigvecs=np.linalg.eig(C)

In [ ]:
# np.linalg.eig does not sort eigenvalues in any particular order, so do this
order = np.flip(eigvals.argsort())
eigvals=eigvals[order]
eigvecs=eigvecs[:,order]

In [ ]:
# Reshape the 1-D eigenvectors to 2-D nlon*nlat so we can plot them
eofscov=np.reshape(eigvecs,(nlat,nlon,nlon*nlat))
eofscov[0:10,0,0]

In [ ]:
#

In [ ]:
# EOFs are left singular vectors, U, of Singular Value Decomposition
U,S,Vt=np.linalg.svd(datap2d.T)

In [ ]:
# S from svd are already sorted in descending order (except for the last few 
# that are so small ~1e-14 that they are insignificant)

In [ ]:
eofssvd=np.reshape(U,(nlat,nlon,nlon*nlat))

In [ ]:
eofssvd[0:10,0,0]

In [ ]:
# Mode 1
ax=plt.pcolormesh(ds.longitude,ds.latitude,eofssvd[:,:,0],cmap='RdBu')
plt.colorbar(ax)

In [ ]:
# Mode 2
ax=plt.pcolormesh(ds.longitude,ds.latitude,eofssvd[:,:,1],cmap='RdBu')
plt.colorbar(ax)

In [ ]:
# Verify that SVD and COV modes are the same
ax=plt.pcolormesh(ds.longitude,ds.latitude,eofscov[:,:,1],cmap='RdBu')
plt.colorbar(ax)